## VGG_BPCA
Here we are using a VGG net with 19 layers with the dataset cifar 100.

We are trying to reproduce the results of the paper [1] changing the max pooling layers for blocked based principal component analysis (BPCA) layers like the article [2] proposes.

[1] https://arxiv.org/pdf/1706.05350.pdf

[2] http://www.ic.uff.br/iwssip2010/Proceedings/nav/papers/paper_10.pdf

[3] https://ieeexplore.ieee.org/document/5654484

## Imports

In [12]:
%pip install lime
import os
import time

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import tensorflow as tf

from keras import backend
from keras.applications import imagenet_utils
from keras.callbacks import ModelCheckpoint
from keras.datasets import cifar100
from keras.engine import training
from keras.layers import VersionAwareLayers
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import data_utils, layer_utils, to_categorical
from sklearn.model_selection import train_test_split
# from google.colab import drive


Note: you may need to restart the kernel to use updated packages.


In [13]:
np.random.seed(77)
tf.random.set_seed(77)


In [14]:
classes_dict = {
    "0": ['apple'],
    "1": ['aquarium_fish'],
    "2": ['baby'],
    "3": ['bear'],
    "4": ['beaver'],
    "5": ['bed'],
    "6": ['bee'],
    "7": ['beetle'],
    "8": ['bicycle'],
    "9": ['bottle'],
    "10": ['bowl'],
    "11": ['boy'],
    "12": ['bridge'],
    "13": ['bus'],
    "14": ['butterfly'],
    "15": ['camel'],
    "16": ['can'],
    "17": ['castle'],
    "18": ['caterpillar'],
    "19": ['cattle'],
    "20": ['chair'],
    "21": ['chimpanzee'],
    "22": ['clock'],
    "23": ['cloud'],
    "24": ['cockroach'],
    "25": ['couch'],
    "26": ['crab'],
    "27": ['crocodile'],
    "28": ['cup'],
    "29": ['dinosaur'],
    "30": ['dolphin'],
    "31": ['elephant'],
    "32": ['flatfish'],
    "33": ['forest'],
    "34": ['fox'],
    "35": ['girl'],
    "36": ['hamster'],
    "37": ['house'],
    "38": ['kangaroo'],
    "39": ['computer_keyboard'],
    "40": ['lamp'],
    "41": ['lawn_mower'],
    "42": ['leopard'],
    "43": ['lion'],
    "44": ['lizard'],
    "45": ['lobster'],
    "46": ['man'],
    "47": ['maple_tree'],
    "48": ['motorcycle'],
    "49": ['mountain'],
    "50": ['mouse'],
    "51": ['mushroom'],
    "52": ['oak_tree'],
    "53": ['orange'],
    "54": ['orchid'],
    "55": ['otter'],
    "56": ['palm_tree'],
    "57": ['pear'],
    "58": ['pickup_truck'],
    "59": ['pine_tree'],
    "60": ['plain'],
    "61": ['plate'],
    "62": ['poppy'],
    "63": ['porcupine'],
    "64": ['possum'],
    "65": ['rabbit'],
    "66": ['raccoon'],
    "67": ['ray'],
    "68": ['road'],
    "69": ['rocket'],
    "70": ['rose'],
    "71": ['sea'],
    "72": ['seal'],
    "73": ['shark'],
    "74": ['shrew'],
    "75": ['skunk'],
    "76": ['skyscraper'],
    "77": ['snail'],
    "78": ['snake'],
    "79": ['spider'],
    "80": ['squirrel'],
    "81": ['streetcar'],
    "82": ['sunflower'],
    "83": ['sweet_pepper'],
    "84": ['table'],
    "85": ['tank'],
    "86": ['telephone'],
    "87": ['television'],
    "88": ['tiger'],
    "89": ['tractor'],
    "90": ['train'],
    "91": ['trout'],
    "92": ['tulip'],
    "93": ['turtle'],
    "94": ['wardrobe'],
    "95": ['whale'],
    "96": ['willow_tree'],
    "97": ['wolf'],
    "98": ['woman'],
    "99": ['worm']
}

labels = [classes_dict[str(i)][0] for i in range(100)]


# Load Data

In [15]:
(X_train, y_train), (X_test, y_test) = cifar100.load_data()
X_train = X_train / 255.0
X_test = X_test / 255.0

X_test, X_valid, y_test, y_valid = train_test_split(
    X_test,
    y_test,
    test_size=0.7
)

y_train = to_categorical(y_train, 100)
y_test = to_categorical(y_test, 100)
y_valid = to_categorical(y_valid, 100)

datagen = ImageDataGenerator(
    height_shift_range=0.25,
    width_shift_range=0.25,
    shear_range=0.25,
    zoom_range=0.25,
    rotation_range=45,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)


# BPCA

In [16]:

class BPCALayer(tf.keras.layers.Layer):
    def __init__(self, pool_size=2, stride=2, n_components=1):
        super(BPCALayer, self).__init__()
        self.pool_size = pool_size
        self.stride = stride
        self.n_components = n_components

    def call(self, inputs):
        def bpca_pooling(feature_map):
            # print(feature_map.shape)

            # Compute the region of interest
            feature_map_height = int(feature_map.shape[0])
            feature_map_width = int(feature_map.shape[1])
            feature_map_channels = int(feature_map.shape[2])

            h = feature_map_height
            w = feature_map_width
            c = feature_map_channels

            # Create blocks (patches)
            data = tf.reshape(
                feature_map,
                [1, feature_map_height, feature_map_width, feature_map_channels]
            )
            pool_size = self.pool_size
            strides = self.stride

            patch_size = [1, pool_size, pool_size, 1]
            strides = [1, strides, strides, 1]
            patches = tf.image.extract_patches(
                images=data,
                sizes=patch_size,
                strides=strides,
                rates=[1, 1, 1, 1],
                padding='VALID'
            )
            d = c // (self.pool_size * self.pool_size)
            data = tf.reshape(
                patches,
                [h*w*d, self.pool_size * self.pool_size]
            )
            # data = tf.reshape(patches, [h*w*8, 4])

            # Normalize the data by subtracting the mean and dividing by the standard deviation
            mean = tf.reduce_mean(data, axis=0)
            std = tf.math.reduce_std(data, axis=0)
            data = (data - mean) / std

            data = tf.where(tf.math.is_nan(data), 0.0, data)

            # Perform the Singular Value Decomposition (SVD) on the data
            s, u, v = tf.linalg.svd(data)

            # Extract the first n principal components from the matrix v
            pca_components = v[:, :self.n_components]

            # Perform the PCA transformation on the data
            transformed_data = tf.matmul(data, pca_components)

            return tf.reshape(transformed_data, [h // self.pool_size, w // self.pool_size, feature_map_channels])

        pooled = tf.map_fn(bpca_pooling, inputs, dtype=tf.float32)
        return pooled


# VGG16

In [17]:
layers = VersionAwareLayers()
WEIGHTS_PATH = (
    "https://storage.googleapis.com/tensorflow/keras-applications/"
    "vgg16/vgg16_weights_tf_dim_ordering_tf_kernels.h5"
)
WEIGHTS_PATH_NO_TOP = (
    "https://storage.googleapis.com/tensorflow/"
    "keras-applications/vgg16/"
    "vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
)


def VGG16(include_top=True, weights="imagenet", input_tensor=None, input_shape=None, pooling=None, classes=1000, classifier_activation="softmax"):
    if not (weights in {"imagenet", None} or tf.io.gfile.exists(weights)):
        raise ValueError(
            "The `weights` argument should be either "
            "`None` (random initialization), `imagenet` "
            "(pre-training on ImageNet), "
            "or the path to the weights file to be loaded.  Received: "
            f"weights={weights}"
        )

    if weights == "imagenet" and include_top and classes != 1000:
        raise ValueError(
            'If using `weights` as `"imagenet"` with `include_top` '
            "as true, `classes` should be 1000.  "
            f"Received `classes={classes}`"
        )

    input_shape = imagenet_utils.obtain_input_shape(
        input_shape,
        default_size=224,
        min_size=32,
        data_format=backend.image_data_format(),
        require_flatten=include_top,
        weights=weights,
    )

    if input_tensor is None:
        img_input = layers.Input(shape=input_shape)
    else:
        if not backend.is_keras_tensor(input_tensor):
            img_input = layers.Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    # Block 1
    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same", name="block1_conv1")(img_input)
    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same", name="block1_conv2")(x)
    # x = layers.MaxPooling2D((2, 2), strides=(2, 2), name="block1_pool")(x)
    # x = layers.AveragePooling2D((2, 2), strides=(2, 2), name="block1_pool")(x)
    x = BPCALayer(pool_size=2, stride=2, n_components=1)(x)

    # Block 2
    x = layers.Conv2D(128, (3, 3), activation="relu", padding="same", name="block2_conv1")(x)
    x = layers.Conv2D(128, (3, 3), activation="relu", padding="same", name="block2_conv2")(x)
    # x = layers.MaxPooling2D((2, 2), strides=(2, 2), name="block2_pool")(x)
    # x = layers.AveragePooling2D((2, 2), strides=(2, 2), name="block2_pool")(x)
    x = BPCALayer(pool_size=2, stride=2, n_components=1)(x)

    # Block 3
    x = layers.Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv1")(x)
    x = layers.Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv2")(x)
    x = layers.Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv3")(x)
    # x = layers.MaxPooling2D((2, 2), strides=(2, 2), name="block3_pool")(x)
    # x = layers.AveragePooling2D((2, 2), strides=(2, 2), name="block3_pool")(x)
    x = BPCALayer(pool_size=2, stride=2, n_components=1)(x)

    # Block 4
    x = layers.Conv2D(512, (3, 3), activation="relu", padding="same", name="block4_conv1")(x)
    x = layers.Conv2D(512, (3, 3), activation="relu", padding="same", name="block4_conv2")(x)
    x = layers.Conv2D(512, (3, 3), activation="relu", padding="same", name="block4_conv3")(x)
    # x = layers.MaxPooling2D((2, 2), strides=(2, 2), name="block4_pool")(x)
    # x = layers.AveragePooling2D((2, 2), strides=(2, 2), name="block4_pool")(x)
    x = BPCALayer(pool_size=2, stride=2, n_components=1)(x)

    # Block 5
    x = layers.Conv2D(512, (3, 3), activation="relu", padding="same", name="block5_conv1")(x)
    x = layers.Conv2D(512, (3, 3), activation="relu", padding="same", name="block5_conv2")(x)
    x = layers.Conv2D(512, (3, 3), activation="relu", padding="same", name="block5_conv3")(x)
    # x = layers.MaxPooling2D((2, 2), strides=(2, 2), name="block5_pool")(x)
    # x = layers.AveragePooling2D((2, 2), strides=(2, 2), name="block5_pool")(x)
    x = BPCALayer(pool_size=2, stride=2, n_components=1)(x)

    if include_top:
        # Classification block
        x = layers.Flatten(name="flatten")(x)
        x = layers.Dense(4096, activation="relu", name="fc1")(x)
        x = layers.Dense(4096, activation="relu", name="fc2")(x)

        imagenet_utils.validate_activation(classifier_activation, weights)
        x = layers.Dense(
            classes, activation=classifier_activation, name="predictions"
        )(x)
    else:
        if pooling == "avg":
            x = layers.GlobalAveragePooling2D()(x)
        elif pooling == "max":
            x = layers.GlobalMaxPooling2D()(x)
        else:
            x = BPCALayer(pool_size=2, stride=2, n_components=1)(x)

    if input_tensor is not None:
        inputs = layer_utils.get_source_inputs(input_tensor)
    else:
        inputs = img_input

    vgg16_model = training.Model(inputs, x, name="vgg16")

    if weights == "imagenet":
        if include_top:
            weights_path = data_utils.get_file(
                "vgg16_weights_tf_dim_ordering_tf_kernels.h5",
                WEIGHTS_PATH,
                cache_subdir="models",
                file_hash="64373286793e3c8b2b4e3219cbf3544b",
            )
        else:
            weights_path = data_utils.get_file(
                "vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5",
                WEIGHTS_PATH_NO_TOP,
                cache_subdir="models",
                file_hash="6d6bbae143d832006294945121d1f1fc",
            )
        vgg16_model.load_weights(weights_path)
    elif weights is not None:
        vgg16_model.load_weights(weights)

    return vgg16_model


In [18]:
# VGG16
vgg16_model = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=X_train[0].shape,
    classes=len(labels),
    pooling="max"
)

vgg16_model.summary()


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Instructions for updating:
Use fn_output_signature instead
(32, 32, 64)
(16, 16, 128)
(8, 8, 256)
(4, 4, 512)
(2, 2, 512)
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 bpca_layer (BPCALayer)      (None, 16, 16, 64)        0         
                                                                 
 b

In [19]:
model = Sequential()
model.add(vgg16_model)
# model.add(layers.Flatten(name="flatten"))
model.add(layers.Dense(4096, activation="relu", name="fc1"))
model.add(layers.Dropout(0.35))
model.add(layers.Dense(4096, activation="relu", name="fc2"))
model.add(layers.Dense(len(labels), activation='softmax', name="predictions"))

untrainable_layers = [0]
for idx, layer in enumerate(model.layers):
    print(idx, layer)
    layer.trainable = False
    if idx not in untrainable_layers:
        layer.trainable = True

model.summary()
vgg16_model = model
del model


(32, 32, 64)
(16, 16, 128)
(8, 8, 256)
(4, 4, 512)
(2, 2, 512)
0 <keras.engine.functional.Functional object at 0x7fcdfe418130>
1 <keras.layers.core.dense.Dense object at 0x7fcdfe3de0a0>
2 <keras.layers.regularization.dropout.Dropout object at 0x7fcdfe414b50>
3 <keras.layers.core.dense.Dense object at 0x7fcdfe485220>
4 <keras.layers.core.dense.Dense object at 0x7fcdfe48f370>
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 fc1 (Dense)                 (None, 4096)              2101248   
                                                                 
 dropout_1 (Dropout)         (None, 4096)              0         
                                                                 
 fc2 (Dense)                 (None, 4096)              16781312  
       

# Training

In [20]:
model_checkpointer = ModelCheckpoint(
    f'cifar100_vgg16_bpca_warmup.h5',
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='max'
)

optimizer = Adam(learning_rate=1e-5)

vgg16_model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)

start_time = time.perf_counter()
history = vgg16_model.fit(
    datagen.flow(X_train, y_train, batch_size=128, shuffle=True),
    validation_data=(X_valid, y_valid),
    epochs=10,
    verbose=1,
    callbacks=[
        model_checkpointer
    ]
)
end_time = time.perf_counter()


Epoch 1/10
(32, 32, 64)
(16, 16, 128)
(8, 8, 256)
(4, 4, 512)
(2, 2, 512)
(32, 32, 64)
(16, 16, 128)
(8, 8, 256)
(4, 4, 512)
(2, 2, 512)
 34/391 [=>............................] - ETA: 9:28 - loss: 4.6744 - accuracy: 0.0083

KeyboardInterrupt: 